# Obesity Modelado

**Variable objetivo:** Obesity: 
- **0** -> Riesgo nulo o bajo
- **1** -> Riesgo medio
- **2** -> Riesgo alto

Se trata de un problema de clasificación supervisada. 

**Tareas:**

1. Cargar los datos
2. Selección y separación del dataset
3. Transformación de los datos (si necesario)
4. Entrenar y evaluar diferentes modelos de clasificación
5. Escoger el mejor modelo
6. Ajustar y mejorar el modelo escogido
7. Contruir el pipeline definitivo


In [1]:
# Librerías
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt # importar altair con el alias alt

# Dividir el dataframe
from sklearn.model_selection import train_test_split

# Transformar columnas
#from sklearn.impute import SimpleImputer
#from sklearn.preprocessing import OneHotEncoder, Binarizer, StandardScaler, RobustScaler
#from sklearn.compose import ColumnTransformer

# Clasificación supervisada
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Métricas
from sklearn.metrics import accuracy_score, classification_report

# Armando pipelines
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline


# Funciones propias
import data_common_functions as dcf
import char_common_functions as ccf

### Cargar los datos

In [2]:
# Leemos el archivo
CURRENT_DIR = Path.cwd()

BASE_DIR = Path(CURRENT_DIR).parent

df_obesity = pd.read_parquet(f"{BASE_DIR}/data/out/obesity_labeled.parquet", engine='fastparquet')

df_obesity

,age,age_range,gender,height,weight,waist_circum_preferred,hip_circum,gender_bin,bmi,rcc,...,obesity_cc,obesity_cc_txt,obesity_rcc,obesity_rcc_txt,obesity_ict,obesity_ict_txt,risk_factors,cluster_kmeans,cluster,obesity
1,47.0,46-55,male,187.1980,156.630,138.5062,142.1892,1,44.696399,0.974098,...,1,1-Alto,1,1-Medio,3,3-Obesidad,4,5,1,2
2,50.0,46-55,male,180.5940,98.064,98.3996,109.9058,1,30.067891,0.895309,...,1,1-Alto,0,0-Bajo,2,2-Sobrepeso,3,1,4,2
3,28.0,26-35,male,182.7022,89.211,97.0026,107.0102,1,26.725809,0.906480,...,1,1-Alto,0,0-Bajo,2,2-Sobrepeso,3,1,4,2
4,52.0,46-55,male,185.9026,113.273,105.9942,111.0996,1,32.775964,0.954047,...,1,1-Alto,1,1-Medio,2,2-Sobrepeso,4,1,1,2
5,50.0,46-55,male,183.1086,108.279,104.0892,112.3950,1,32.294365,0.926102,...,1,1-Alto,0,0-Bajo,2,2-Sobrepeso,3,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,35.0,26-35,female,152.8064,49.259,61.5950,93.0910,0,21.096116,0.661664,...,0,0-Bajo,0,0-Bajo,0,0-Delgado,0,2,0,0
4461,40.0,36-45,female,168.6052,67.646,73.8124,100.4062,0,23.795794,0.735138,...,0,0-Bajo,0,0-Bajo,1,1-Normal,0,2,5,0
4462,23.0,17-25,female,168.5036,73.775,78.0034,109.7026,0,25.983093,0.711044,...,0,0-Bajo,0,0-Bajo,1,1-Normal,1,0,2,0
4463,24.0,17-25,male,170.5102,65.149,75.7936,97.7900,1,22.408203,0.775065,...,0,0-Bajo,0,0-Bajo,1,1-Normal,0,0,5,0


## Separación del dataset

1. Selección de columnas
2. Separación de la variable objetivo
3. División del dataset

In [3]:
def select_split_dataset(df, cols_to_leave, var_target, train_proportion, test_proportion):
    
    cols_to_leave.append(var_target)
    
    df = ccf.select_data(df, [], cols_to_leave)
    
    obesity_risk = df[var_target].copy()
    obesity_data = df.drop([var_target], axis=1)

    original_count = len(df)
    training_size = int(original_count * train_proportion)
    test_size = int((1 - train_proportion) * test_proportion * training_size)
    
    train_x, rest_x, train_y, rest_y = train_test_split(obesity_data, 
                                                        obesity_risk, 
                                                        train_size=training_size,
                                                        stratify=obesity_risk)
    test_x, validate_x, test_y, validate_y = train_test_split(rest_x, 
                                                              rest_y, 
                                                              train_size=test_size,
                                                              stratify=rest_y)

    return (train_x, train_y), (validate_x, validate_y), (test_x, test_y)
    

In [4]:
cols_to_leave = ['age',
                 'gender_bin',
                 'height',
                 'weight',
                 'waist_circum_preferred',
                 'hip_circum']

training_data, validate_data, test_data = select_split_dataset(df_obesity, cols_to_leave, 'obesity', 
                                                        train_proportion=0.6, test_proportion=0.5)

train_x=training_data[0]
train_y=training_data[1]
validate_x=validate_data[0]
validate_y=validate_data[1]
test_x=test_data[0]
test_y=test_data[1]



In [5]:
test_x.shape

(531, 6)

## Escalado y transformación de datos

In [6]:
cols_encode = []
cols_scale = ['age', 'height', 'weight', 'waist_circum_preferred', 'hip_circum']
cols_pass = ['gender_bin']


X_train_scaled = dcf.transform_df(train_x, cols_encode, cols_scale, cols_pass)
X_validate_scaled = dcf.transform_df(validate_x, cols_encode, cols_scale, cols_pass)
X_test_scaled = dcf.transform_df(test_x, cols_encode, cols_scale, cols_pass)

[[-0.38489719 -0.20127375  0.73833962  0.73199055  0.79642368  1.        ]
 [-0.22878398 -1.74827696 -1.40634228 -1.70909674 -0.99523464  0.        ]
 [ 1.25429158 -1.85632163  0.72601386  1.67620236  2.13591313  0.        ]
 ...
 [-1.55574632  0.16460478 -0.06283466 -0.24781655 -1.30154257  1.        ]
 [ 0.39566889  1.08298447  1.62554768  1.5225806   0.63354565  1.        ]
 [-0.77518023  0.18424927  0.565779    0.45284856  0.30778959  1.        ]]
[[ 0.27759633  0.2487449   0.09175631  0.05697825 -0.61442017  1.        ]
 [ 0.04193    -0.04323751 -0.7508591  -1.10657746 -0.11369095  0.        ]
 [ 0.74892901  1.49326011  4.35368064  4.05646101  3.77947874  1.        ]
 ...
 [-1.52917892 -1.68264696 -1.22711999 -0.87233027 -1.12766763  0.        ]
 [ 1.86441634 -1.37151816 -1.13797372 -1.37922583 -0.77214988  0.        ]
 [ 1.22026168  1.45496733  0.46128591  0.39682868 -0.03357428  1.        ]]
[[ 2.06599229 -1.15957897 -0.57106725 -0.44551765 -0.29672721  0.        ]
 [-0.98911466

In [7]:
X_test_scaled.shape

(531, 6)

## Inicialización de los algoritmos

In [8]:
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
dtree = DecisionTreeClassifier(random_state=42)
svm = SVC(kernel='linear', C=1.0, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

## Evaluación de diferentes modelos

1. Entrenar y evaluar los diferentes modelos
2. Comparar resultados y escoger el mejor

### Entrenamiento

In [9]:
log_reg.fit(X_train_scaled, train_y)
dtree.fit(X_train_scaled, train_y)
svm.fit(X_train_scaled, train_y)
rf.fit(X_train_scaled, train_y)

RandomForestClassifier(random_state=42)

### Validaciones

In [10]:
y_pred_log_reg = log_reg.predict(X_test_scaled)
y_pred_dtree = dtree.predict(X_test_scaled)
y_pred_svm = svm.predict(X_test_scaled)
y_pred_rf = rf.predict(X_test_scaled)

## Evaluación del rendimiento

In [11]:
# Calcular la precisión de cada clasificador
accuracy_log_reg = accuracy_score(test_y, y_pred_log_reg)
accuracy_dtree = accuracy_score(test_y, y_pred_dtree)
accuracy_svm = accuracy_score(test_y, y_pred_svm)
accuracy_rf = accuracy_score(test_y, y_pred_rf)

# Imprimir los resultados
print("Exactitud de Regresión Logística Multinomial:", accuracy_log_reg)
print("Exactitud de Árboles de Decisión:", accuracy_dtree)
print("Exactitud de SVM:", accuracy_svm)
print("Exactitud de Random Forest:", accuracy_rf)

# Informe detallado de la clasificación
print("\nInforme de clasificación para Regresión Logística Multinomial:\n", classification_report(test_y, y_pred_log_reg))
print("Informe de clasificación para Árboles de Decisión:\n", classification_report(test_y, y_pred_dtree))
print("Informe de clasificación para SVM:\n", classification_report(test_y, y_pred_svm))
print("Informe de clasificación para Random Forest:\n", classification_report(test_y, y_pred_rf))


Exactitud de Regresión Logística Multinomial: 0.9152542372881356
Exactitud de Árboles de Decisión: 0.9397363465160076
Exactitud de SVM: 0.9171374764595104
Exactitud de Random Forest: 0.943502824858757

Informe de clasificación para Regresión Logística Multinomial:
               precision    recall  f1-score   support

           0       0.97      0.96      0.96       319
           1       0.54      0.36      0.43        39
           2       0.88      0.97      0.92       173

    accuracy                           0.92       531
   macro avg       0.80      0.76      0.77       531
weighted avg       0.91      0.92      0.91       531

Informe de clasificación para Árboles de Decisión:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97       319
           1       0.69      0.62      0.65        39
           2       0.91      0.98      0.95       173

    accuracy                           0.94       531
   macro avg       0.86      

In [12]:
def test_models(df, params):
    
    var_target = params['var_target']
    cols_leave = params['cols_leave']
    cols_encode = params['cols_encode']
    cols_scale = params['cols_scale']
    cols_pass = params['cols_pass']
    train_proportion = params['train_proportion']
    test_proportion = params['test_proportion']
    
    # Divide el dataframe
    training_data, validate_data, test_data = select_split_dataset(df, cols_leave, var_target, 
                                                                   train_proportion=train_proportion, 
                                                                   test_proportion=test_proportion)

    train_x=training_data[0]
    train_y=training_data[1]
    validate_x=validate_data[0]
    validate_y=validate_data[1]
    test_x=test_data[0]
    test_y=test_data[1]
    
    # Transforma los datos 
    X_train_scaled = dcf.transform_df(train_x, cols_encode, cols_scale, cols_pass)
    X_validate_scaled = dcf.transform_df(validate_x, cols_encode, cols_scale, cols_pass)
    X_test_scaled = dcf.transform_df(test_x, cols_encode, cols_scale, cols_pass)
    
    # Crea los modelos
    log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
    dtree = DecisionTreeClassifier(random_state=42)
    svm = SVC(kernel='linear', C=1.0, random_state=42)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Entrena los modelos
    log_reg.fit(X_train_scaled, train_y)
    dtree.fit(X_train_scaled, train_y)
    svm.fit(X_train_scaled, train_y)
    rf.fit(X_train_scaled, train_y)
    
    # Prueba los modelos entrenados
    y_pred_log_reg = log_reg.predict(X_test_scaled)
    y_pred_dtree = dtree.predict(X_test_scaled)
    y_pred_svm = svm.predict(X_test_scaled)
    y_pred_rf = rf.predict(X_test_scaled)
    
    # Muestra y compara las métricas
    
    # Calcular la precisión de cada clasificador
    accuracy_log_reg = accuracy_score(test_y, y_pred_log_reg)
    accuracy_dtree = accuracy_score(test_y, y_pred_dtree)
    accuracy_svm = accuracy_score(test_y, y_pred_svm)
    accuracy_rf = accuracy_score(test_y, y_pred_rf)

    # Imprimir los resultados
    print("Exactitud de Regresión Logística Multinomial:", accuracy_log_reg)
    print("Exactitud de Árboles de Decisión:", accuracy_dtree)
    print("Exactitud de SVM:", accuracy_svm)
    print("Exactitud de Random Forest:", accuracy_rf)

    # Informe detallado de la clasificación
    print("\nInforme de clasificación para Regresión Logística Multinomial:\n", classification_report(test_y, y_pred_log_reg))
    print("Informe de clasificación para Árboles de Decisión:\n", classification_report(test_y, y_pred_dtree))
    print("Informe de clasificación para SVM:\n", classification_report(test_y, y_pred_svm))
    print("Informe de clasificación para Random Forest:\n", classification_report(test_y, y_pred_rf))


In [13]:
def test_dtree(df, params):
    
    var_target = params['var_target']
    cols_leave = params['cols_leave']
    cols_encode = params['cols_encode']
    cols_scale = params['cols_scale']
    cols_pass = params['cols_pass']
    train_proportion = params['train_proportion']
    test_proportion = params['test_proportion']
    
    # Divide el dataframe
    training_data, validate_data, test_data = select_split_dataset(df, cols_leave, var_target, 
                                                                   train_proportion=train_proportion, 
                                                                   test_proportion=test_proportion)

    train_x=training_data[0]
    train_y=training_data[1]
    validate_x=validate_data[0]
    validate_y=validate_data[1]
    test_x=test_data[0]
    test_y=test_data[1]
    
    # Transforma los datos 
    X_train_scaled = dcf.transform_df(train_x, cols_encode, cols_scale, cols_pass)
    X_validate_scaled = dcf.transform_df(validate_x, cols_encode, cols_scale, cols_pass)
    X_test_scaled = dcf.transform_df(test_x, cols_encode, cols_scale, cols_pass)
    
    # Crea los modelos
    dtree = DecisionTreeClassifier(random_state=42)
    
    # Entrena los modelos
    dtree.fit(X_train_scaled, train_y)
    
    # Prueba con los datos de test
    print("\nPrueba con test:\n")
    
    # Prueba los modelos entrenados
    y_pred_dtree = dtree.predict(X_test_scaled)
    
    # Muestra y compara las métricas
    
    # Calcular la precisión de cada clasificador
    accuracy_dtree = accuracy_score(test_y, y_pred_dtree)

    # Imprimir los resultados
    print("Exactitud de Árboles de Decisión:", accuracy_dtree)


    # Informe detallado de la clasificación
    print("\nInforme de clasificación para Árboles de Decisión:\n", classification_report(test_y, y_pred_dtree))

    
    # Prueba con los datos de test
    print("\nPrueba con validate:\n")
    
    # Prueba los modelos entrenados
    y_pred_validate_dtree = dtree.predict(X_validate_scaled)
    
    # Muestra y compara las métricas
    
    # Calcular la precisión de cada clasificador
    accuracy_validate_dtree = accuracy_score(validate_y, y_pred_validate_dtree)

    # Imprimir los resultados
    print("Exactitud de Árboles de Decisión:", accuracy_validate_dtree)


    # Informe detallado de la clasificación
    print("\nInforme de clasificación para Árboles de Decisión:\n", classification_report(validate_y, y_pred_validate_dtree))


In [14]:
def test_rf(df, params):
    
    var_target = params['var_target']
    cols_leave = params['cols_leave']
    cols_encode = params['cols_encode']
    cols_scale = params['cols_scale']
    cols_pass = params['cols_pass']
    train_proportion = params['train_proportion']
    test_proportion = params['test_proportion']
    
    # Divide el dataframe
    training_data, validate_data, test_data = select_split_dataset(df, cols_leave, var_target, 
                                                                   train_proportion=train_proportion, 
                                                                   test_proportion=test_proportion)

    train_x=training_data[0]
    train_y=training_data[1]
    validate_x=validate_data[0]
    validate_y=validate_data[1]
    test_x=test_data[0]
    test_y=test_data[1]
    
    # Transforma los datos 
    X_train_scaled = dcf.transform_df(train_x, cols_encode, cols_scale, cols_pass)
    X_validate_scaled = dcf.transform_df(validate_x, cols_encode, cols_scale, cols_pass)
    X_test_scaled = dcf.transform_df(test_x, cols_encode, cols_scale, cols_pass)
    
    # Crea los modelos
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Entrena los modelos
    rf.fit(X_train_scaled, train_y)
    
    # Prueba con los datos de test
    print("\nPrueba con test:\n")
    
    # Prueba los modelos entrenados
    y_pred_rf = rf.predict(X_test_scaled)
    
    # Muestra y compara las métricas
    
    # Calcular la precisión de cada clasificador
    accuracy_rf = accuracy_score(test_y, y_pred_rf)

    # Imprimir los resultados
    print("Exactitud de Random Forest:", accuracy_rf)


    # Informe detallado de la clasificación
    print("\nInforme de clasificación para Random Forest:\n", classification_report(test_y, y_pred_rf))

    
    # Prueba con los datos de test
    print("\nPrueba con validate:\n")
    
    # Prueba los modelos entrenados
    y_pred_validate_rf = rf.predict(X_validate_scaled)
    
    # Muestra y compara las métricas
    
    # Calcular la precisión de cada clasificador
    accuracy_validate_rf = accuracy_score(validate_y, y_pred_validate_rf)

    # Imprimir los resultados
    print("Exactitud de Random Forest:", accuracy_validate_rf)


    # Informe detallado de la clasificación
    print("\nInforme de clasificación para Random Forest:\n", classification_report(validate_y, y_pred_validate_rf))


## Pruebas con diferentes columnas

In [15]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'height', 'weight', 'waist_circum_preferred', 'hip_circum'],
    'cols_encode': [],
    'cols_scale': ['age', 'height', 'weight', 'waist_circum_preferred', 'hip_circum'],
    'cols_pass': ['gender_bin'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_models(df_obesity, params)

[[-0.93541776 -0.11803596 -0.22645751 -0.81643687  0.58208565  0.        ]
 [-1.32698963 -0.50283011  0.13440152 -0.33442208  1.03298022  0.        ]
 [ 1.02244159 -0.59106316 -0.9855058  -1.04704787 -0.5364318   0.        ]
 ...
 [ 1.02244159 -0.73566733  0.19661859  0.03795798  0.95575518  0.        ]
 [ 1.10075597 -0.43665532  0.77822379  1.06625621  0.92586162  0.        ]
 [-0.07395965 -0.1376433   1.88892298  0.95473122  2.789227    0.        ]]
[[-0.42342406 -0.15554364 -0.92094633 -1.28738396 -0.4250758   0.        ]
 [-0.9221344   0.71629352 -0.85375696 -0.19742755 -0.84459369  1.        ]
 [-0.03380661  0.74330819  3.14511813  2.71681217  1.74901397  1.        ]
 ...
 [-0.96109614 -0.41586685  0.2316098   0.57534578  0.4386375   1.        ]
 [ 1.68051016  0.07776489 -0.26874182  0.18895911 -0.78536764  1.        ]
 [-0.18965359  0.98644025  0.51365749  0.43886094 -0.30909144  1.        ]]
[[-0.44257881  1.88425122  1.08990981  0.99824715  0.52526895  1.        ]
 [-0.67819291

In [16]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'waist_circum_preferred', 'bmi', 'rcc', 'ict'],
    'cols_encode': [],
    'cols_scale': ['age', 'waist_circum_preferred', 'bmi', 'rcc', 'ict'],
    'cols_pass': ['gender_bin'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_models(df_obesity, params)

[[-0.91480852 -0.01062518  0.34947837 -0.25149983  0.12651582  1.        ]
 [ 0.97457135 -0.81659526 -0.69530069 -0.97113334 -1.05607119  0.        ]
 [-1.07225684 -0.01062518 -0.93797711  0.51176872 -0.37748332  0.        ]
 ...
 [ 0.89584719  1.15012599  0.67009821  1.00211893  0.66857882  1.        ]
 [-0.83608436 -1.71897325 -0.70051583 -2.07488302 -1.36441096  0.        ]
 [ 0.10860558 -1.07304029 -0.49554082 -0.5804149  -0.31096613  0.        ]]
[[-0.31277699 -0.16546777 -0.77137134 -0.48565322 -0.31622928  0.        ]
 [ 0.61989856  2.25249522  2.8607113   0.36547493  2.41545157  0.        ]
 [-1.24545255 -0.65661651 -0.59653032  0.11846423 -0.04980615  0.        ]
 ...
 [ 0.69762153  0.00643428 -0.39755917  0.3801659   0.26155492  0.        ]
 [ 1.39712819  1.18896931  0.67587926  1.14244023  0.80598509  1.        ]
 [-1.32317551 -0.41859827 -1.15616724 -0.15689373 -1.16242312  1.        ]]
[[-0.99122287 -0.53793352 -0.71685416  0.02875459 -0.87722158  1.        ]
 [ 0.31737049

In [17]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'waist_circum_preferred', 'bmi', 'rcc', 'ict'],
    'cols_encode': [],
    'cols_scale': ['age', 'waist_circum_preferred'],
    'cols_pass': ['gender_bin', 'bmi', 'rcc', 'ict'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_models(df_obesity, params)

[[ 0.9550738   0.13654136  1.         25.5035238   0.88984599  0.48867546]
 [ 0.64292441 -1.17152712  0.         25.63122531  0.67626614  0.45249169]
 [-0.91782257 -1.39685863  0.         22.11325992  0.67061245  0.41319169]
 ...
 [-0.60567317  0.31795232  0.         27.62376712  0.83567851  0.52604714]
 [ 0.09666297 -0.10979562  0.         27.8149432   0.76995305  0.50461538]
 [-1.12071967 -0.70367781  1.         19.70821012  0.86889084  0.42227279]]
[[ 0.79299752  1.12421082  0.         33.3215739   0.89589977  0.59963409]
 [ 0.23961603 -0.34995419  1.         23.32878976  0.82052618  0.44922989]
 [ 0.16056153  0.68742118  0.         27.72402241  0.95435792  0.56529708]
 ...
 [-0.62998345 -0.68507727  0.         19.65119624  0.81513988  0.43584103]
 [-0.15565646  0.22050813  0.         22.07434058  0.91349206  0.54748692]
 [-1.34147393  0.30899568  1.         18.76591016  1.02880658  0.5195962 ]]
[[-0.71892764 -1.03155107  0.         23.27210338  0.69539078  0.41426653]
 [-0.10055787

In [18]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict'],
    'cols_encode': [],
    'cols_scale': [],
    'cols_pass': ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_models(df_obesity, params)

[[21.  0.  0.  1.  0.  1.]
 [27.  1.  0.  1.  0.  1.]
 [47.  1.  0.  1.  0.  1.]
 ...
 [28.  1.  1.  2.  0.  2.]
 [35.  0.  1.  3.  1.  3.]
 [38.  0.  1.  3.  0.  3.]]
[[46.  0.  1.  1.  2.  2.]
 [35.  0.  0.  1.  0.  0.]
 [27.  0.  0.  1.  0.  1.]
 ...
 [40.  0.  1.  1.  1.  2.]
 [20.  1.  0.  1.  0.  1.]
 [59.  1.  1.  1.  2.  2.]]
[[32.   0.   1.   2.   2.   2. ]
 [34.   1.   0.   1.   0.   1. ]
 [44.   0.   1.   2.   1.   2. ]
 ...
 [22.   0.   0.   1.   0.   2. ]
 [51.   1.   0.   1.   0.   1. ]
 [53.4  0.   1.   2.   2.   3. ]]
Exactitud de Regresión Logística Multinomial: 0.9905838041431262
Exactitud de Árboles de Decisión: 0.9962335216572504
Exactitud de SVM: 0.9849340866290018
Exactitud de Random Forest: 0.9981167608286252

Informe de clasificación para Regresión Logística Multinomial:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       319
           1       1.00      0.87      0.93        39
           2       0.99      1

In [19]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict',
                   'height', 'weight', 'waist_circum_preferred', 'hip_circum'],
    'cols_encode': [],
    'cols_scale': ['height', 'weight', 'waist_circum_preferred', 'hip_circum'],
    'cols_pass': ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_dtree(df_obesity, params)

[[-1.42349759 -1.39526008 -1.43511982 ...  1.          0.
   1.        ]
 [ 1.11678577  0.5209886   0.57029819 ...  2.          0.
   1.        ]
 [-0.48327853  0.31464938  0.2164009  ...  2.          0.
   2.        ]
 ...
 [ 0.98384991 -0.20216514 -0.34534084 ...  1.          0.
   1.        ]
 [ 0.19348582  0.05346355  0.8212095  ...  2.          1.
   2.        ]
 [-0.26333012 -0.17969494 -0.03451041 ...  2.          1.
   2.        ]]
[[-0.38464592 -0.83082202 -0.41706762 ...  1.          1.
   1.        ]
 [ 1.55328273  0.75099978  0.64549784 ...  2.          0.
   1.        ]
 [ 1.42575452  0.10346295 -0.67928451 ...  1.          0.
   0.        ]
 ...
 [-0.17960056 -1.11146372 -1.25655304 ...  1.          0.
   0.        ]
 [ 0.78561293 -0.612008   -0.57165818 ...  1.          0.
   1.        ]
 [-0.10958508 -0.875857   -0.57165818 ...  1.          0.
   1.        ]]
[[-9.18019167e-01 -7.09034321e-04  5.71567610e-01 ...  2.00000000e+00
   1.00000000e+00  2.00000000e+00]
 [ 4.23

In [20]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'height', 'weight', 'waist_circum_preferred', 'hip_circum'],
    'cols_encode': [],
    'cols_scale': ['age', 'height', 'weight', 'waist_circum_preferred', 'hip_circum'],
    'cols_pass': ['gender_bin'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_dtree(df_obesity, params)

[[ 1.34871583 -0.48041096  1.52265399  1.80693376  0.72892087  1.        ]
 [ 1.66434866 -0.83281466 -0.84968289 -1.27108324 -0.52196876  0.        ]
 [ 0.79635838  0.68103222  2.26859496  2.30032736  1.8063473   1.        ]
 ...
 [-0.86071399  0.25169532 -0.60511208 -0.4038491  -0.94463264  1.        ]
 [ 1.56176799 -0.99908964 -0.14311782  0.26792526  0.97812154  0.        ]
 [-0.81336906 -0.70624713 -1.08129145 -1.06803279 -1.01304067  0.        ]]
[[ 0.76400191 -0.68065629 -0.79733386 -1.20466471 -0.50626383  0.        ]
 [-0.39229676 -0.96942696  0.34683206 -0.14105359  0.50844621  1.        ]
 [-1.10920194  0.28743556 -1.35874635 -0.68130051 -1.40660607  1.        ]
 ...
 [ 1.99738715  0.66141725  1.5230094   1.72730034  0.98781761  1.        ]
 [ 0.04709673 -0.69012418  0.32549111  0.2641316   1.3430878   0.        ]
 [-1.3173357  -0.06524338  0.52871515  0.51549648  0.37461155  1.        ]]
[[-1.14028938  1.14224483  0.09149267 -0.46886351 -0.36775107  1.        ]
 [ 1.58874007

In [21]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict'],
    'cols_encode': [],
    'cols_scale': [],
    'cols_pass': ['gender_bin', 'age', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_dtree(df_obesity, params)

[[ 0. 52.  1.  2.  0.  2.]
 [ 0. 58.  1.  2.  2.  3.]
 [ 0. 35.  0.  1.  0.  1.]
 ...
 [ 0. 41.  1.  3.  1.  3.]
 [ 0. 46.  0.  1.  0.  1.]
 [ 1. 44.  1.  2.  0.  1.]]
[[ 1. 42.  0.  2.  0.  2.]
 [ 0. 22.  0.  0.  0.  0.]
 [ 0. 24.  0.  2.  0.  2.]
 ...
 [ 1. 22.  0.  1.  0.  1.]
 [ 1. 63.  1.  3.  1.  2.]
 [ 0. 44.  0.  1.  0.  1.]]
[[ 0.  35.   1.   3.   1.   3. ]
 [ 1.  30.   0.   2.   0.   1. ]
 [ 0.  22.   1.   1.   2.   2. ]
 ...
 [ 0.  47.   0.   1.   0.   1. ]
 [ 0.  22.2  0.   1.   0.   1. ]
 [ 0.  25.   0.   1.   0.   0. ]]

Prueba con test:

Exactitud de Árboles de Decisión: 1.0

Informe de clasificación para Árboles de Decisión:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       319
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00       173

    accuracy                           1.00       531
   macro avg       1.00      1.00      1.00       531
weighted avg       1.00 

In [22]:
params = {
    'var_target': 'obesity',
    'cols_leave': ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict'],
    'cols_encode': [],
    'cols_scale': [],
    'cols_pass': ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict'],
    'train_proportion': 0.6,
    'test_proportion': 0.5
}

test_rf(df_obesity, params)

[[29.4  1.   1.   3.   0.   2. ]
 [48.   1.   0.   1.   0.   1. ]
 [32.   0.   0.   1.   0.   1. ]
 ...
 [53.   0.   0.   2.   0.   2. ]
 [31.   1.   0.   3.   0.   1. ]
 [47.   1.   1.   3.   0.   2. ]]
[[43.5  0.   0.   1.   0.   1. ]
 [55.   1.   1.   1.   1.   1. ]
 [24.   1.   1.   2.   0.   1. ]
 ...
 [40.   1.   0.   1.   0.   0. ]
 [27.   1.   0.   3.   0.   2. ]
 [43.   1.   1.   3.   0.   2. ]]
[[27.6  0.   0.   1.   0.   1. ]
 [60.   0.   0.   1.   1.   1. ]
 [65.   1.   0.   1.   0.   1. ]
 ...
 [23.   0.   0.   1.   0.   1. ]
 [56.4  1.   0.   1.   0.   1. ]
 [29.9  1.   0.   2.   0.   2. ]]

Prueba con test:

Exactitud de Random Forest: 1.0

Informe de clasificación para Random Forest:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       319
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00       173

    accuracy                           1.00       531
   macro avg      

### Conclusión

Usaré los árboles de decisión.

Ahora debo construir el pipeline para poder exportar el modelo:

A partir de los datos: edad, género, peso, altura, circunferencia de cintura y de cadera:

1. Calcular los indicadores
2. Seleccionar los datos que necesitamos
3. Transformar los datos
4. Hacer la predicción
5. Mostrar los resultados


### Todavía no tengo muy claro si se guarda todo el proceso, incluso el entrenamiento del modelo. Eso debo revisarlo

Creo que haré un nuevo notebook donde construiré el pipeline final. Este quedaría para el estudio de cómo seleccioné el modelo. Ver los notebooks de Antonio para hacer el último notebook


## Main


In [23]:
def save_model():
    
    from joblib import dump
    import pickle
    
    CURRENT_DIR = Path.cwd()

    BASE_DIR = Path(CURRENT_DIR).parent

    df_obesity = pd.read_parquet(f"{BASE_DIR}/data/out/obesity_labeled.parquet", engine='fastparquet')
    
    cols_to_leave = ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict', 'obesity']
      
    df_train = ccf.select_data(df_obesity, [], cols_to_leave)
    
    Y_train = df_train['obesity'].copy()
    obesity_data = df_train.drop(['obesity'], axis=1)
    
    cols_encode = []
    cols_scale = []
    cols_pass = ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict']
    
    # Transformo los datos 
    X_train_scaled = dcf.transform_df(obesity_data, cols_encode, cols_scale, cols_pass)

    # Creo el modelo
    model = DecisionTreeClassifier(random_state=42)
    
    # Entreno los modelos
    model.fit(X_train_scaled, Y_train)
    
    dump(model, 'dtree_obesity.joblib')

    # save the model to a file
    with open(f"{BASE_DIR}/models/dtree_obesity.pkl", "wb") as f:
        pickle.dump(model, f)
    

In [24]:
save_model()

[[47.  1.  1.  3.  1.  3.]
 [50.  1.  1.  3.  0.  2.]
 [28.  1.  1.  2.  0.  2.]
 ...
 [23.  0.  0.  2.  0.  1.]
 [24.  1.  0.  1.  0.  1.]
 [22.  0.  0.  1.  0.  0.]]


### Pipeline para predicción para una persona

**Entrada:** Un JSON con la siguiente información para una persona: Edad, género, peso, estatura, contorno de cintura y contorno de cadera

**Salida:** Indicadores calculados y variable **obesity**: 

**Pasos:**

1. Calcular las nuevas variables
2. Seleccionar sólo las columnas que nos interesan
3. Hacer la predicción con el modelo ya entrenado
4. Devolver el resultado

In [25]:
person = [{
    'age': 49,
    'gender': 'female',
    'height': 154.01,
    'weight': 51.20,
    'waist_circum_preferred': 85.0,
    'hip_circum': 96
}]

json_object = json.dumps(person) 
df_person = pd.read_json(json_object, orient='records')
df_person

,age,gender,height,weight,waist_circum_preferred,hip_circum
0,49,female,154.01,51.2,85,96


In [26]:
from custom_functions import custom_calculus
import pickle

CURRENT_DIR = Path.cwd()
BASE_DIR = Path(CURRENT_DIR).parent

df_person_transformed = custom_calculus(df_person)

cols_to_leave = ['age', 'gender_bin', 'obesity_cc', 'obesity_bmi', 'obesity_rcc', 'obesity_ict']
      
df_person_transformed = ccf.select_data(df_person_transformed, [], cols_to_leave)

with open(f"{BASE_DIR}/models/dtree_obesity.pkl", "rb") as f:
    trained_model = pickle.load(f)

df_person_transformed['obesity'] = trained_model.predict(df_person_transformed) 

df_person_transformed.columns = ['age', 'gender', 'cc', 'bmi', 'rcc', 'ict', 'obesity']

df_person_transformed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4430 entries, 1 to 4464
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     4430 non-null   float64
 1   age_range               4430 non-null   object 
 2   gender                  4430 non-null   object 
 3   height                  4430 non-null   float64
 4   weight                  4430 non-null   float64
 5   waist_circum_preferred  4430 non-null   float64
 6   hip_circum              4430 non-null   float64
 7   gender_bin              4430 non-null   int64  
 8   bmi                     4430 non-null   float64
 9   rcc                     4430 non-null   float64
 10  ict                     4430 non-null   float64
 11  obesity_bmi             4430 non-null   int64  
 12  obesity_bmi_txt         4430 non-null   object 
 13  obesity_cc              4430 non-null   int64  
 14  obesity_cc_txt          4430 non-null   

/home/diana/anaconda3/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


,age,gender,cc,bmi,rcc,ict,obesity
0,49,0,1,1,2,2,2


In [27]:
js = df_person_transformed.to_json(orient = 'records')

print(js)

[{"age":49,"gender":0,"cc":1,"bmi":1,"rcc":2,"ict":2,"obesity":2}]


In [28]:
diccionario = {'age': 0, 'gender': 'female', 'weight': 0.0, 'height': 0.0, 'waist_circum_preferred': 0.0, 'hip_circum': 0.0}

df_dic = pd.DataFrame.from_dict([diccionario])

df_dic

,age,gender,weight,height,waist_circum_preferred,hip_circum
0,0,female,0.0,0.0,0.0,0.0
